1. Add dependencies

In [1]:
@file:DependsOn("dev.langchain4j:langchain4j:1.6.0")
@file:DependsOn("dev.langchain4j:langchain4j-open-ai:1.6.0")
@file:DependsOn("dev.langchain4j:langchain4j-http-client-jdk:1.6.0")

2. Set up models under test and utilities

In [2]:
import dev.langchain4j.http.client.jdk.JdkHttpClient
import dev.langchain4j.model.openai.OpenAiChatModel
import java.net.http.HttpClient
import java.time.Duration

val durationLimit = Duration.ofMinutes(10)
val httpClientBuilder = HttpClient.newBuilder()
    .version(HttpClient.Version.HTTP_1_1)
    .connectTimeout(durationLimit)

val jdkHttpClientBuilder = JdkHttpClient.builder()
    .httpClientBuilder(httpClientBuilder)
    .connectTimeout(durationLimit)
    .readTimeout(durationLimit)

val openAiModel = OpenAiChatModel.builder()
    .baseUrl("http://127.0.0.1:1234/v1")
    .httpClientBuilder(jdkHttpClientBuilder)
    .timeout(durationLimit)
    .temperature(0.0)
    .returnThinking(false)
    .modelName("openai/gpt-oss-20b")
    .build()

val answer = openAiModel.chat("What is the capital city of Japan?")
println(answer)


The capital city of Japan is **Tokyo**.


Service Test

In [3]:
import dev.langchain4j.service.AiServices
import dev.langchain4j.service.Result

interface CodeGenAiService {
    fun generateCode(prompt: String): Result<String>
}

interface TranslatorAiService {
    fun translate(message: String): Result<String>
}

val noThink =
"""
    /no_think
    thinking_enabled: false
    detailed thinking off
"""
.trimIndent()

fun executeCodeGen(model: ChatModel) {
    val codeGenService = AiServices.builder(CodeGenAiService::class.java)
        .systemMessageProvider { "convert this to Kotlin" + noThink }
        .chatModel(model)
        .build()

    val result = codeGenService.generateCode("System.out.println(\"Hello, world!\"")
    println(result.content())
    println("\n")
    println("*".repeat(100))
    println("\n")
}

fun executeTranslation(model: ChatModel) {
    val translatorAiService = AiServices.builder(TranslatorAiService::class.java)
        .systemMessageProvider { "translate this to Bahasa Indonesia." + noThink }
        .chatModel(model)
        .build()

    val result = translatorAiService.translate("Good morning")
    println(result.finalResponse().aiMessage().text())

    println("\n")
    println("*".repeat(100))
    println("\n")
}

executeCodeGen(openAiModel)
executeTranslation(openAiModel)

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[3], line 20, column 27: Unresolved reference: ChatModel
at Cell In[3], line 33, column 31: Unresolved reference: ChatModel